In [1]:
import qutip, pickle, sys
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import time as time
import math, cmath
import Max_Ent_aux_lib as me
#import proj_ev_library as projev
#import max_entev library as meev
from IPython.display import display, Math, Latex

np.set_printoptions(threshold=1.e-3,linewidth=120,precision=1, formatter={"float":lambda x: str(.001*int(1000*x)) })

Tenemos la siguiente ecuación de movimiento:

$$
    \dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{\bigg(c_{M+2}(t) - c_M(t) \bigg)\langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle + \bigg(c_{M-2}(t) - c_M(t)\bigg) \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle + p \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle \bigg\}
$$

siendo $I_{M}^{0}(t) = \langle \rho_{M}^{0\dagger}(t) \rho_{M}^{0}(t)\rangle = I_{-M}^{0}(t)$
la cual puede reescribirse como 

$$
\dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{A_{\rho^{M, M+2}} c_{M+2}(t) - \bigg(A_{\rho^{M, M+2}} + B_{\rho^{M, M-2}} \bigg) c_{M}(t) + B_{\rho^{M, M-2}} c_{M-2}(t) \bigg\} + p C_{\rho^M} \\
= - i{\bf{\mathcal{M}}}(t) \textbf{c}(t)
$$

donde 

<ol>

<li> $A_{\rho^{M, M+2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle$ </li> 
<li> $B_{\rho^{M, M-2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle$ </li>
<li> $C_{\rho^M} = \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle$ </li>
    
</ol>    

Entonces, si $f(t) = \langle \rho(t) \rho^0(t) \rangle = \sum_{M} \dot{c}_M(t) I_{M}^{0}(t)$

$$
    \dot{f}(t) = \sum_{M} \bigg(\dot{c}_M(t) I_{M}^{0}(t) + {c}_M(t) \dot{I}_{M}^{0}(t) \bigg)
$$

where

1: $A=\alpha B = N(t) \exp(-m t^{1+a}) = I_M^{(0)}$ (a primer orden alpha = 1) \
2: $C = M N(t) exp(-m t^{1+a})$

con $N(t) = 1/Tr(\rho(t))$ la calculo con la definición del $\rho_M (t)=  c_{M}(t) \rho^0_M(t)$, with $c_0(t)$= 1. 
Note that the kernel $K(t,t') = e^{(-i M(t-t'))}$ is not a solution to the previous differential equation for the M-tensor is time-dependent
    
$H_ {0, 2}^\dagger = H_{0, -2}$ 

$\rho_{M}^\dagger = rho{-M}$

## Step 1: Fix parameters and initial conditions for the coherences

In [59]:
p = .0009;    # strength of the Sigma Interaction Hamiltonian
a = .05;      # Power-law factor
M = 3;      # Truncation/Total no. of coherences
coherences_t0_pert0 = 1. # Unused for the time being
param_list = {"total_no_cohrs": M, "p_factor": p, "power_law_factor": a} # dictionary containing the simulation's initial 
                                                                         # parameters 

coeff_list_t0 = np.array([coherences_t0_pert0 - np.random.rand() for i in range(param_list["total_no_cohrs"])])
              # initial configuration for the coherences, random numbers for the time being. Can be changed in the future

coeff_list_t0 = coeff_list_t0/(sum(coeff_list_t0)) 
              # normalization of the initial vector, so that Tr c_0 = 1. 

## Step 1. bis: A simple verification for the hermitian-ness and general form of the M-tensor

In [19]:
M_tensor_f= np.array(me.gen_func_even_cohr_M_matrix(param_list, coeff_list_t0, 1, False, False, False, False))
qutip.Qobj(M_tensor_f)
     #* TODO: not hermitian, fix later. 

Quantum object: dims = [[5], [5]], shape = (5, 5), type = oper, isherm = False
Qobj data =
[[0.11900000000000001 0.88 0.0 0.0 0.0]
 [0.156 -0.312 1.156 0.0 0.0]
 [0.0 0.156 -0.312 1.156 0.0]
 [0.0 0.0 0.156 -0.312 1.156]
 [0.0 0.0 0.0 0.499 0.5]]

In [20]:
M_tensor_f @ coeff_list_t0

array([0.187, 0.08, 0.23800000000000002, 0.454, 0.322])

## Step 2: Setting up a real-valued system of M x M real differential equations

In [21]:
### Already included in the accompanying library 

from scipy.integrate import odeint

def real_differential_system(cohr_vector_t0, t, parameters):
    """
    This module sets up the system of time-dependent differential
    equations, given by 
    
    dC/dt = M(t)*c(t),
    
    wherein M(t) is the coherence-weight matrix. 
    In accordance with Scipy.Odeint's documentation, this module s0peeftakes
    as input the following parameters, 
    
    *** 1. cohr_vector_t0: an initial configuration of weights for the coherences at time zero, 
           such that this configuration has trace one.
    *** 2. t: time
    *** 3. parameters: a dictionary containing three parameters:
                        ☼*☼: the total number of coherences in the system,
                        ☼*☼: the chosen p-factor, the weight for the 
                             perturbation Hamiltonian, Sigma. 
                        ☼*☼: the power law factor used in the approximation.
    
    ===> Returns: an array of numbers
    
    """
    return (me.gen_func_even_cohr_M_matrix(parameters = param_list,
                                       init_configurations = cohr_vector_t0, 
                                       timet = t,
                                       closed_boundary_conditions = False,
                                       visualization = False,
                                       as_qutip_qobj = False,
                                          add_mxm_dimensional_zeros = False)) @ (cohr_vector_t0)

## Step 2. bis: Simple test to ensure the function works as intended 

In [22]:
A = real_differential_system(cohr_vector_t0 = coeff_list_t0, 
                    t=2, parameters = param_list
                    )
A

array([0.185, 0.077, 0.24, 0.456, 0.322])

## Step 3: solving said system of MxM differential equations

In [23]:
from scipy.integrate import odeint

ts = np.linspace(0.1, 1., 100)              ## times 
result = odeint(func = real_differential_system, 
                y0 = coeff_list_t0, 
                t = ts,
                args = ((param_list,)))
result

array([[0.022, 0.209, 0.123, 0.211, 0.433],
       [0.023, 0.21, 0.125, 0.215, 0.436],
       [0.025, 0.211, 0.127, 0.219, 0.439],
       ...,
       [0.267, 0.449, 0.549, 0.774, 0.917],
       [0.271, 0.454, 0.557, 0.783, 0.925],
       [0.275, 0.459, 0.5640000000000001, 0.791, 0.933]])

In [24]:
len(result[:, 0])

100

In [25]:
len(result[0])

5

In [26]:
n_dim_zero_array = [0 for n in range(M)]
len(n_dim_zero_array)

5

## Complex differential equation custom solver

In [60]:
def Mtensor_2mx2m_dimensional_symplectic(parameters, init_configurations, timet, 
                                            closed_boundary_conditions = False,
                                            visualization = False,
                                            as_qutip_qobj = False):
    """
    This module constructs the even-coherences weight matrix,
    M, wherein M_{m, m'} is the weight of the m-m' coherence 
    term, with both m and m' even numbered-coherences. The odd
    coherences are disregardes for these do not factor into the 
    M-matrix. It takes the following as input:
    ***. a dictionary of parameters
         **. where, in said dictionary,
             1. the total number of even coherences to be 
                considered is explicited,
             2. the strength of the Sigma-interaction 
                Hamiltonian, labelled p,
             3. and the power law factor for its submodules.
    ***. a mesh for the times, 
    ***. a boolean option, not implemented as of yet. 
    ***. a boolean option for visualizing a
             1. plot of the M-matrix's eigenvalues,
             2. and the M-matrix coefficients in a Hinton
                diagram.
    ***. a boolean option for returning the M-matrix as 
         quantum object.     
    
        ===> Returns a real-valued matrix. 
    *Note that his module does not return a sparse matrix. 
    """
    M = parameters["total_no_cohrs"]; p = parameters["p_factor"]; a = parameters["power_law_factor"]
    m_matrix_list = []; t = timet;
    
    m_dimensional_zero_array = [0 for m in range(M)]
    
    for m in range(M):
        if m == 0:
            local_array = np.array(m_dimensional_zero_array + [me.A_mmplustwo_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a) + 
                                           p * me.C_m_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a)] 
                                          + [me.A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, 
                                                                   power_law_factor = a)] 
                                          + [0 for i in range(M-2)])
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
        if (m > 0 and m < M-1):
            list_with_zeros = [0 for i in range(m-1)]
            local_array = np.array(m_dimensional_zero_array + list_with_zeros
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)]
                                          + [me.diag_mm_matrix_elmt(cohrnc = m, time = t, power_law_factor = a, p = p)] 
                                          + [me.A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, power_law_factor = a)]  
                                          + [0 for i in range(M - (len(list_with_zeros)+3))])
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None       
        if (m == M-1):
            local_array = np.array(m_dimensional_zero_array + [0 for i in range(M - 2)] 
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)] 
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a) + 
                                               p * me.C_m_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)])
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
            
    for m in range(M):
        if m == 0:
            local_array = np.array([me.A_mmplustwo_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a) + 
                                           p * me.C_m_matrix_elmt(cohrnc = m, time = t, 
                                                                   power_law_factor = a)] 
                                          + [me.A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, 
                                                                   power_law_factor = a)] 
                                          + [0 for i in range(M-2)]
                                + m_dimensional_zero_array)
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
        if (m > 0 and m < M-1):
            list_with_zeros = [0 for i in range(m-1)]
            local_array = np.array(list_with_zeros
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)]
                                          + [me.diag_mm_matrix_elmt(cohrnc = m, time = t, power_law_factor = a, p = p)] 
                                          + [me.A_mmplustwo_matrix_elmt(cohrnc = m+2, time = t, power_law_factor = a)]  
                                          + [0 for i in range(M - (len(list_with_zeros)+3))]
                                + m_dimensional_zero_array)
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None       
        if (m == M-1):
            local_array = np.array([0 for i in range(M - 2)] 
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)] 
                                          + [me.B_mmminustwo_matrix_elmt(cohrnc = m, time = t, power_law_factor = a) + 
                                               p * me.C_m_matrix_elmt(cohrnc = m, time = t, power_law_factor = a)]
                                + m_dimensional_zero_array)
            local_array = local_array/(sum(local_array))
            m_matrix_list.append(local_array)
            local_array = None
    
    return m_matrix_list

In [67]:

ts = np.linspace(0.1, 1., 100)
y0 = coeff_list_t0

def complex_differential_system(cohr_complex, t, parameters):
    Mtensor = Mtensor_2mx2m_dimensional_symplectic(parameters = parameters, init_configurations = cohr_complex, timet = t, 
                                                closed_boundary_conditions = False,
                                                visualization = False,
                                                as_qutip_qobj = False)
    return Mtensor @ cohr_complex

from scipy.integrate import odeint

cohr_complex_t0 = [coherences_t0_pert0 - np.random.rand() for i in range(param_list["total_no_cohrs"])]
cohr_complex_t0 += [0 for i in range(param_list["total_no_cohrs"])]
cohr_complex_t0 = np.array(cohr_complex_t0)


In [70]:
B = complex_differential_system(cohr_complex = cohr_complex_t0, t=1, parameters = param_list)
len(B)

6

In [71]:
ts = np.linspace(0.1, 1., 100)              ## times 
result = odeint(func = complex_differential_system, 
                y0 = cohr_complex_t0, 
                t = ts,
                args = ((param_list,)))
result

array([[0.8160000000000001, 0.6880000000000001, 0.896, 0.0, 0.0, 0.0],
       [0.8160000000000001, 0.6880000000000001, 0.896, 0.006, 0.008, 0.007],
       [0.8160000000000001, 0.6880000000000001, 0.896, 0.012, 0.017, 0.014],
       ...,
       [1.195, 0.993, 1.2550000000000001, 0.709, 0.9410000000000001, 0.793],
       [1.203, 0.999, 1.2630000000000001, 0.718, 0.9530000000000001, 0.804],
       [1.212, 1.006, 1.2710000000000001, 0.727, 0.965, 0.8140000000000001]])

In [72]:
len(result[:, 0])

100

In [73]:
len(result[0])

6

# Tests y fallos :(

In [23]:
Preguntas:
    
    1. La matrix M tiene que depender con el tiempo me parece, si A, B y C lo hacen: Check 
    2. Como tendría que implementar la norma???: Check
    3. Empezar a jugar con los parámetros: Checkn't
    
m+2 vs m+2 : Check
** reescribir la matriz en tèrminos de los c_pares. : Check

SyntaxError: invalid syntax (3991330021.py, line 1)

In [45]:
def generating_function_even_cohr_M_matrix(parameters, init_configuration, timespan, 
                                           closed_boundary_conditions = False):
    
    M = parameters["total_no_cohrs"]; p = parameters["p_factor"]; a = parameters["power_law_factor"]
    cm_list = init_configuration
    m_matrix_list = []; t = timespan;
    
    for m in range(M):
        if m == 0:
            m_matrix_list.append(np.array([diag_mm_matrix_elmt(m,t,a)] 
                                    + [A_mmplustwo_matrix_elmt(m+2,t,a)] + [0 for k in range(int((M-3)/2))])) 
        if m == 1:
            local_list = [0] + [diag_mm_matrix_elmt(m,t,a)] + [A_mmplustwo_matrix_elmt(m+2, t, a)]
            local_length = len(np.array(local_list))
            m_matrix_list.append(np.array(local_list + [0 for k in range(int(M/2 + 1)-local_length)]))
        if (m > 1) and (m < M - 2):  
            local_list = [0 for j in range(m-2)]; local_length = len(local_list)
            local_list += ([B_mmminustwo_matrix_elmt(-m-2,t,a)] + [diag_mm_matrix_elmt(m,t,a)] 
                        + [A_mmplustwo_matrix_elmt(m+2,t,a)])
            local_list += [0 for j in range(int(M/2+1)-local_length - 5)]
            m_matrix_list.append(np.array(local_list))
        if m == M - 2: 
            m_matrix_list.append(np.array([0 for i in range(int(M/2)-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)]  
                                 + [diag_mm_matrix_elmt(m,t,a)] + [0]))
        if m == M - 1:
            m_matrix_list.append(np.array([0 for i in range(int(M/2)-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)] + [0] 
                                 + [diag_mm_matrix_elmt(m,t,a)]))
    return (m_matrix_list)


In [72]:
v = [np.random.rand() for i in range(5)]
def example_matrix(t, p):
    return (t+p) * np.identity(5) 

def example_diff_eq(v, t, p):
    return example_matrix(t, p).dot(v)

ts = np.linspace(0., 10., 100)
result = odeint(func = example_diff_eq, 
                y0 = v, 
                t = ts,
                args = (p,))

In [4]:
A = np.array([-1,2,3])
sum(A)

4

In [ ]:
# Si la matrix M(t) = M para todo t, uso la propiedad de semigrupo y ya fue

def semigroup_phit_and_rhot_sol(coeff_list_t0, Mtensor, ts):
    coeff_at_timet_list = []
    coeff_at_timet_list.append(np.array(coeff_list_t0))
    
    for i in range(1, len(ts)-1):
        try: 
            a = (ts[i+1] * Mtensor).expm() * coeff_list_t0[0]
            coeff_at_timet_list.append(a)
        #if (Exception):
        #   continue 
    return coeff_at_timet_list    

# Si la matrix M(t) fuese no constante, odeints

from scipy.integrate import ode
import random

def F(y, t, Mtensor):
    return Mtensor.dot(y)

real_Mtensor = (np.real(Mtensor))
ts1 = np.linspace(0, 500, 200)
result = odeint(func=F, y0=coeff_list_t0, t= ts1 , args=(real_Mtensor,))
plt.figure(figsize=(7, 4))

for i in range(len(total_no_spins)):
    plt.plot(ts1, result[:,i], label=r'$c_{(%d)}(t)$'%i)

plt.xlabel("t")
plt.ylabel("Phi(t)")
plt.legend()

